In [1]:
include("./src/REoptLite.jl")
using JuMP
using Xpress
using Main.REoptLite
using JSON


┌ Info: Xpress: Found license file C:\xpressmp\bin\xpauth.xpr
└ @ Xpress C:\Users\jchang2\.julia\packages\Xpress\kcAft\src\license.jl:42
┌ Info: Xpress: Development license detected.
└ @ Xpress C:\Users\jchang2\.julia\packages\Xpress\kcAft\src\license.jl:84


In [2]:
function run(scenario, fname)
    
    m = Model(optimizer_with_attributes(Xpress.Optimizer, "logfile" => "output.log"))
    results = run_reopt(m, REoptInputs(scenario))
    
    open(fname*".json", "w") do f
        JSON.print(f, results)
    end
    
    return results
end

run (generic function with 1 method)

In [3]:
scenario_template = JSON.parsefile("test/scenarios/STM_campus_existing_sseb_crit.json")

Dict{String,Any} with 8 entries:
  "ElectricUtility" => Dict{String,Any}("outage_durations"=>Any[336],"outage_st…
  "ElectricTariff"  => Dict{String,Any}("urdb_label"=>"5e611fec5457a3665e019406…
  "Site"            => Dict{String,Any}("latitude"=>39.7407,"longitude"=>-105.1…
  "Generator"       => Dict{String,Any}("max_kw"=>0.0,"only_runs_during_grid_ou…
  "ElectricLoad"    => Dict{String,Any}("loads_kw"=>Any[4323.64, 4308.39, 4295.…
  "PV"              => Dict{String,Any}("array_type"=>0,"existing_kw"=>3580.54)
  "Storage"         => Dict{String,Any}("macrs_option_years"=>5,"can_grid_charg…
  "Financial"       => Dict{String,Any}("owner_discount_pct"=>0.1,"offtaker_dis…

In [4]:
results = Dict[]

for VoLL in range(0, stop=1000)
    scenario_dict = copy(scenario_template)
    scenario_dict["ElectricUtility"]["outage_start_timesteps"] = collect(range(17, stop=8760-336, step=336))
    scenario_dict["Financial"]["VoLL"] = collect(range(VoLL, stop=1000, step=20))
    s = Scenario(scenario_dict)
    push!(results, run(s, "sseb_VoLL"*string(VoLL)))
    println("run $VoLL complete")
end

┌ Info: Checking URDB for 
│   urdb_label = 5e611fec5457a3665e019406
└ @ Main.REoptLite C:\Users\jchang2\Documents\REoptLite\src\core\urdb.jl:151
┌ Info: Querying PVWatts for prodfactor with 
│   pv.name = PV
└ @ Main.REoptLite C:\Users\jchang2\Documents\REoptLite\src\core\prodfactor.jl:41
┌ Info: PVWatts success.
└ @ Main.REoptLite C:\Users\jchang2\Documents\REoptLite\src\core\prodfactor.jl:49


MethodError: MethodError: Cannot `convert` an object of type Array{Int64,1} to an object of type Real
Closest candidates are:
  convert(::Type{T}, !Matched::T) where T<:Number at number.jl:6
  convert(::Type{T}, !Matched::Number) where T<:Number at number.jl:7
  convert(::Type{T}, !Matched::Base.TwicePrecision) where T<:Number at twiceprecision.jl:250
  ...

In [5]:
r = results

0-element Array{Dict,1}